###### BUILD THE NEURAL NETWORK

Neural networks comprise of layers/modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let’s check to see if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we continue to use the CPU.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


__Define the Class__

We define our neural network by subclassing <code>nn.Module</code>, and initialize the neural network layers in <code>\_\_init\_\_</code>. Every <code>nn.Module</code> subclass implements the operations on input data in the <code>forward<code> method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# create an instance of NeuralNetwork, and move it to the device, and print its structure.
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
# To use the model, we pass it the input data. 
# This executes the model’s forward, along with some background operations. 
# Do not call model.forward() directly!

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='cuda:0')


__Model Layers__

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


__nn.Flatten__

We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


__nn.Linear__

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


__nn.ReLU__

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [12]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1419, -0.1237, -0.0051, -0.3112, -0.3210, -0.3334,  0.4492, -0.0365,
         -0.2299, -0.0812, -0.1491,  0.2570, -0.0641, -0.2245, -0.0198, -0.1678,
          0.0064,  0.0269,  0.3086,  0.4993],
        [ 0.2484,  0.1256,  0.0160,  0.0894, -0.5247, -0.0290,  0.7226, -0.0218,
         -0.2546,  0.2816,  0.0335,  0.0043, -0.0239, -0.1265,  0.0556, -0.0505,
          0.1257, -0.2516,  0.1566,  0.3813],
        [-0.3043, -0.0040, -0.0828, -0.0103, -0.2329, -0.2911,  0.8309,  0.0756,
         -0.0787,  0.0410, -0.2677, -0.1252,  0.3448, -0.0707, -0.5467,  0.0769,
         -0.1376, -0.0346,  0.3932,  0.5178]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4492, 0.0000, 0.0000,
         0.0000, 0.0000, 0.2570, 0.0000, 0.0000, 0.0000, 0.0000, 0.0064, 0.0269,
         0.3086, 0.4993],
        [0.2484, 0.1256, 0.0160, 0.0894, 0.0000, 0.0000, 0.7226, 0.0000, 0.0000,
         0.2816, 0.0335, 0.0043, 0.0000, 0.0000, 0.055

__nn.Sequential__

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like <code>seq_modules</code>.

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

__nn.Softmax__

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. <code>dim</code> parameter indicates the dimension along which the values must sum to 1.

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

__Model Parameters__

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing <code>nn.Module</code> automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s <code>parameters()</code> or <code>named_parameters()</code> methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-5.0085e-03,  8.8300e-06,  1.5156e-02,  ..., -9.6620e-03,
          4.8130e-04,  3.1854e-03],
        [ 1.9945e-02,  3.1282e-02,  1.3897e-02,  ...,  1.5761e-02,
          6.4198e-03,  2.8268e-02]], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0195,  0.0141], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0130, -0.0231,  0.0193,  ...,  0.0196,  0.0041,  0.0009],
        [-0.0026, -0.0334, -0.0

__Further reading:__ [torch.nn API](https://pytorch.org/docs/stable/nn.html)